In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, Operator, Pauli
import warnings
warnings.filterwarnings('ignore')
import uuid


In [3]:

##############################################################################
# 1. Create PARTIALLY entangled state:
#    cos(theta/2)|00> + sin(theta/2)|11>
#THE FUCKING CIRCUIT CONSTRUCTION MF STABBED ME IN THE BACK
##############################################################################
def create_entangled_state(theta):
    qc = QuantumCircuit(2)
    qc.ry(theta, 0)  
    qc.cx(0, 1)
    return qc

##############################################################################
# 2. Measure <Z \otimes Z> after local R_y() rotations on each qubit
##############################################################################
def measure_correlation(state, angle_qubit_0, angle_qubit_1):
    """
    Rotate qubit 0 by 'angle_qubit_0' and qubit 1 by 'angle_qubit_1'
    around Y, then measure Z \otimes Z. Return real number in [-1, +1].
    """
    # R_y(angle) = [[ cos(angle/2), -sin(angle/2)],
    #                [ sin(angle/2),  cos(angle/2)]]
    rotation_0 = Operator([
        [np.cos(angle_qubit_0/2), -np.sin(angle_qubit_0/2)],
        [np.sin(angle_qubit_0/2),  np.cos(angle_qubit_0/2)]
    ])
    rotation_1 = Operator([
        [np.cos(angle_qubit_1/2), -np.sin(angle_qubit_1/2)],
        [np.sin(angle_qubit_1/2),  np.cos(angle_qubit_1/2)]
    ])
    
    # Apply local R_y to each qubit index
    rotated_state = state.evolve(rotation_0, [0]).evolve(rotation_1, [1])
    
    # Now measure Pauli('ZZ')
    correlation = np.real(rotated_state.expectation_value(Pauli('ZZ')))
    return correlation

##############################################################################
# 3. Compute CHSH S-value
##############################################################################
def chsh_s_value(state, angles):
    """
    angles = {
      'A1': float,
      'A2': float,
      'B1': float,
      'B2': float
    }
    S = | <A1 B1> + <A1 B2> + <A2 B1> - <A2 B2> |
    """
    A1B1 = measure_correlation(state, angles['A1'], angles['B1'])
    A1B2 = measure_correlation(state, angles['A1'], angles['B2'])
    A2B1 = measure_correlation(state, angles['A2'], angles['B1'])
    A2B2 = measure_correlation(state, angles['A2'], angles['B2'])
    return abs(A1B1 + A1B2 + A2B1 - A2B2)

##############################################################################


if __name__ == "__main__":
    thetas = np.linspace(0, 2*np.pi, 20)
    print(f"Total thetas: {len(thetas)}")

    backend = Aer.get_backend('statevector_simulator')
    circuits = []

    for idx, theta in enumerate(thetas):
        qc = create_entangled_state(theta)
        qc.name = f"theta_{idx}_{uuid.uuid4()}"
        qc.save_statevector(label=f"state_{idx}")
        circuits.append(qc)

    transpiled_circuits = transpile(circuits, backend=backend)
    job = backend.run(transpiled_circuits)
    result = job.result()

    #prepare angle values for scanning
    #     (We pick a1_values and b1_values, and let A2 = A1 +/- pi/2, B2 = B1 +/- pi/2)
    a1_values = np.linspace(0, 2*np.pi, 15)
    b1_values = np.linspace(0, 2*np.pi, 15)

    max_s_list = []

    #Loop over each theta -> retrieve state -> find max S
    for idx, theta in enumerate(thetas):
        # Retrieve statevector
        qc_name = transpiled_circuits[idx].name
        sv = result.data(qc_name)[f"state_{idx}"]
        state = Statevector(sv)

        best_s = float('-inf')
        best_angles = None

        #loops over A1, B1, plus the +/- pi/2 offset
        for A1 in a1_values:
            A2_options = [A1 + np.pi/2, A1 - np.pi/2]
            for A2 in A2_options:
                for B1 in b1_values:
                    B2_options = [B1 + np.pi/2, B1 - np.pi/2]
                    for B2 in B2_options:
                        angles = {'A1': A1, 'A2': A2, 'B1': B1, 'B2': B2}
                        s_val = chsh_s_value(state, angles)
                        if s_val > best_s:
                            best_s = s_val
                            best_angles = angles
        
        #SAVE ebst theta
        max_s_list.append({
            'theta': theta,
            'max_S': best_s,
            'angles': best_angles
        })

        print(f"Theta = {theta:.3f}, Max S = {best_s:.3f}, Best Angles = {best_angles}")


Total thetas: 20
Theta = 0.000, Max S = 1.405, Best Angles = {'A1': 0.4487989505128276, 'A2': 2.019595277307724, 'B1': 1.7951958020513104, 'B2': 3.365992128846207}
Theta = 0.331, Max S = 1.862, Best Angles = {'A1': 1.3463968515384828, 'A2': -0.2243994752564138, 'B1': 3.5903916041026207, 'B2': 5.161187930897517}
Theta = 0.661, Max S = 2.268, Best Angles = {'A1': 0.0, 'A2': 1.5707963267948966, 'B1': 0.8975979010256552, 'B2': -0.6731984257692414}
Theta = 0.992, Max S = 2.582, Best Angles = {'A1': 0.8975979010256552, 'A2': -0.6731984257692414, 'B1': 6.283185307179586, 'B2': 7.853981633974483}
Theta = 1.323, Max S = 2.768, Best Angles = {'A1': 5.834386356666759, 'A2': 4.263590029871862, 'B1': 1.7951958020513104, 'B2': 3.365992128846207}
Theta = 1.653, Max S = 2.806, Best Angles = {'A1': 0.0, 'A2': -1.5707963267948966, 'B1': 2.243994752564138, 'B2': 3.8147910793590345}
Theta = 1.984, Max S = 2.692, Best Angles = {'A1': 0.0, 'A2': 1.5707963267948966, 'B1': 0.8975979010256552, 'B2': -0.6731984